In [4]:
!pip install webdriver_manager

  Using cached webdriver_manager-3.5.4-py2.py3-none-any.whl (18 kB)


In [5]:
from bs4 import BeautifulSoup
import selenium
from selenium.webdriver.common.keys import Keys

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By 
from webdriver_manager.chrome import ChromeDriverManager

import requests
import shutil

from re import sub

import pandas as pd

import csv

def camel(s):
  s = sub(r"(_|-)+", " ", s).title().replace(" ", "")
  return ''.join([s[0].lower(), s[1:]])

In [6]:
headers = {'User-Agent':        #search "what is my user agent" on Google
           'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.104 Safari/537.36'}
list_page_url = 'https://www.skyscrapercenter.com/quick-lists#q=&page=1&type=building&status=COM&status=UCT&status=STO&min_year=1885&max_year=9999&region=0&country=0&city=1641'

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get(list_page_url)



====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/100.0.4896.60/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\chadd\.wdm\drivers\chromedriver\win32\100.0.4896.60]


In [48]:
bldg_list = []
url_list = []
h_list = []
fl_list = []
year_list = []
mat_list = []
use_list = []

bldgs = driver.find_elements(By.XPATH, '//td[@class="building-hover"]')
urls = driver.find_elements(By.XPATH, '//td[@class="building-hover"]/a')

for b in range(len(bldgs)):
    bldg_list.append(bldgs[b].text)
    url_list.append(urls[b].get_attribute('href'))
    h_list.append(driver.find_element(By.XPATH, '//table[@id="table-buildings"]/tbody/tr['+str(b+1)+']/td[5]').text)
    fl_list.append(driver.find_element(By.XPATH, '//table[@id="table-buildings"]/tbody/tr['+str(b+1)+']/td[6]').text)
    year_list.append(driver.find_element(By.XPATH, '//table[@id="table-buildings"]/tbody/tr['+str(b+1)+']/td[7]').text)
    mat_list.append(driver.find_element(By.XPATH, '//table[@id="table-buildings"]/tbody/tr['+str(b+1)+']/td[8]').text)
    use_list.append(driver.find_element(By.XPATH, '//table[@id="table-buildings"]/tbody/tr['+str(b+1)+']/td[9]').text)

In [79]:
bldg_dict = pd.DataFrame(zip(bldg_list, url_list, h_list, fl_list, year_list, mat_list, use_list),
                        columns = ['bldg', 'url', 'height_m', 'floor', 'year', 'material', 'use'])

In [82]:
bldg_dict.to_csv('data/scraped_bldg_data.csv')

In [65]:
#scrape building images

for b in range(len(bldgs)):    
    driver.get(url_list[b])
    try:
        bldg_img_url=driver.find_element(By.XPATH, '//div[@id="drawing"]/div/div/div/div/img').get_attribute('src')
        filename = 'bldg_img/'+f'{b:02}'+'_'+camel(bldg_list[b])+'.png'
        r = requests.get(bldg_img_url, stream=True)
        if r.status_code == 200:
            with open(filename, 'wb') as f:
                r.raw.decode_content = True
                shutil.copyfileobj(r.raw, f)
    except:
        pass